# ML Coding Challenge
## Random Forest Rangers
## Urban, Atilla, Jano

ssh -N -T -L 30001:localhost:30001 urban@dgx-2-02.ics.unisg.ch

## Setup

In [1]:
import os


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load Training & Test Data

In [2]:
# file locations
DATA_PATH = "drive/MyDrive/AIML24"
TRAINING_PATH = DATA_PATH + "/remote_sensing/otherDatasets/sentinel_2/tif"
TEST_PATH = DATA_PATH + "/remote_sensing/otherDatasets/sentinel_2/testset"

# mount g-drive
try:
    from google.colab import drive
    drive.mount('/content/drive')

    # copy files into runtime for performance
    !cp "drive/MyDrive/AIML24" "AIML24"
    DATA_PATH = "AIML24"
except:
    pass # probably local environment



Mounted at /content/drive
cp: -r not specified; omitting directory 'drive/MyDrive/AIML24'


In [3]:
# Download Training Data
# !wget https://madm.dfki.de/files/sentinel/EuroSATallBands.zip --no-check-certificate
# !unzip EuroSATallBands.zip
# !mv ds/* "drive/MyDrive/AIML24"

# ☝️Test data must be downloaded manually: put into AIML24/remote_sensing/otherDatasets/sentinel_2/testdata dir

In [4]:
# !unzip "drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/testset.zip"
# !mv testset/testset/* "drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/testset"

## Define PyTorch Dataset

In [5]:
import glob
len(glob.glob(os.path.join(TEST_PATH,  f"*.npy")))

4232

In [6]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 67.0 MB/s eta 0:00:00


In [7]:
import rasterio as rio
import glob
from torch.utils.data import Dataset
from collections.abc import Callable
import torch

def load_img(img_path:str) -> np.ndarray:
  if img_path.split('.')[-1] == "tif":
    with rio.open(img_path, "r") as d:
      img = d.read([1,2,3,4,5,6,7,8,9,10,11,12,13])
      img = reshape_as_image(img)
  else:
    img = np.load(img_path)
  return img


ids2labels = {0: 'AnnualCrop',
 1: 'Forest',
 2: 'HerbaceousVegetation',
 3: 'Highway',
 4: 'Industrial',
 5: 'Pasture',
 6: 'PermanentCrop',
 7: 'Residential',
 8: 'River',
 9: 'SeaLake'}

labels2ids = {v: k for k, v in ids2labels.items()}

def text2oh(label):
    one_hot = np.zeros(10)
    one_hot.put(labels2ids[label], 1)
    return one_hot

def oh2text(one_hot):
    try:
        one_hot = one_hot.numpy()
    except:
        pass
    idx = np.argmax(one_hot)
    return ids2labels[idx]

class SentinelTrain(Dataset):
    def __init__(self, transformations=None):
        self.img_paths = [path.replace('\\', '/') for path in glob.glob(os.path.join(TRAINING_PATH, "*", f"*.tif"))]

        # labels = set([path.split('/')[-1].split('_')[0] for path in self.img_paths]) # get unique labels
        # self.label2ids = {name: id for (id, name) in enumerate(sorted(labels))} # map an id to each label
        # self.ids2label = {v:k for k, v in self.label2ids.items()} # reverse lookup dict
        self.transformations = transformations

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = load_img(img_path)

        if self.transformations:
            image = self.transformations(image)

        label = img_path.split("/")[-1].split("_")[0]
        one_hot = text2oh(label)
        # return image, one_hot
        channels = torch.tensor([c for c in range(12)])
        return image, {"ID": one_hot, "channels": channels}



def get_id(img_path):
    return img_path.split("/")[-1].split("_")[-1].split(".")[0]

class SentinelTest(Dataset):
    def __init__(self, transformations=None):
        self.img_paths = [path.replace("\\","/") for path in glob.glob(os.path.join(TEST_PATH,  f"*.npy"))]
        self.transformations = transformations
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = load_img(img_path)
        image_id = get_id(img_path)

        if self.transformations:
            image = self.transformations(image)
        return image, image_id


In [8]:
# Calc train dataset statistics for use in normalization transform

# Create a data loader for your dataset
# data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# # Compute the mean and std of the data
# mean = 0.
# std = 0.
# nb_samples = 0.
# for data in tqdm(data_loader):
#     batch_samples = data[0].size(0)
#     data = data[0].view(batch_samples, data[0].size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     nb_samples += batch_samples

# mean /= nb_samples
# std /= nb_samples

# print("Mean:", mean)
# print("Std:", std)




### Tranforms

In [9]:
# Create instances of the Dataset Class for both train & test
import torch
import torchvision.transforms as transforms
from torchvision import transforms

def bandselect(img):
    return img[:, :, [3,2,1]]

def convert_to_float(img):
    return img.astype(np.float32) / 10000.0

def bands_to_front(img):
    return img.permute(2, 0, 1)

# TODO: improvement -> find global max / min
def l2a_approx(img):
    l2a_bands = img[:,:,[0,1,2,3,4,5,6,7,12,8,10,11]]
    return l2a_bands
    # band_min = np.min(l2a_bands, (0,1)) # minimal value per band
    # return l2a_bands - band_min # dark object subtraction algo approximation

mean = [0.0117, 0.0252, 0.0351, 0.0476, 0.0456, 0.0787, 0.0985, 0.1248, 0.1104,
        0.0204, 0.0810, 0.0598]
std= [0.0065, 0.0154, 0.0188, 0.0278, 0.0228, 0.0356, 0.0456, 0.0531, 0.0502,
        0.0099, 0.0378, 0.0303]

train_transforms = transforms.Compose([
    l2a_approx,
    convert_to_float,
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.GaussianBlur(1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])

train_dataset = SentinelTrain(train_transforms)


test_transforms  = transforms.Compose([
    convert_to_float,
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.GaussianBlur(1),
])

test_dataset = SentinelTest(test_transforms)

print(len(train_dataset))
print(len(test_dataset))

27000
4232




### Dataloaders (adjusted)

In [10]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import numpy as np
import torch


train_dataset_subset, val_dataset_subset = random_split(train_dataset, [.8, .2])

train_dataloader = DataLoader(train_dataset_subset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset_subset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)



## Utils

In [11]:
# Utility function to display image from dataset
from rasterio.plot import reshape_as_image

def normalize_for_display(band_data):
    """Normalize multi-spectral imagery across bands.
    The input is expected to be in HxWxC format, e.g. 64x64x13.
    To account for outliers (e.g. extremly high values due to
    reflective surfaces), we normalize with the 2- and 98-percentiles
    instead of minimum and maximum of each band.
    """
    band_data = np.array(band_data)
    lower_perc = np.percentile(band_data, 2, axis=(0,1))
    upper_perc = np.percentile(band_data, 98, axis=(0,1))
    print(lower_perc)
    print(upper_perc)
    return (band_data - lower_perc) / (upper_perc - lower_perc)


def print_image(img, label, rgb_bands=[3,2,1])-> None:
  """Displays an image. Indices of bands given by "rgb_bands" will be displayed as RGB in the print
  """
  img = img.numpy()
  img = img.transpose(1,2,0)
  img = normalize_for_display(img)
  img = img[:, :, rgb_bands]
  fig, ax = plt.subplots(1, figsize=(5,5))
  ax.imshow((img * 255).astype(np.uint8), vmin=0, vmax=255)

  label = oh2text(label["ID"])
  ax.set_title(label)
  ax.axis(False)
  plt.tight_layout()

  plt.show()

In [12]:
# # example image from test
# img, label = train_dataset.__getitem__(800)
# print_image(img, label, [0,1,2])
# # img.shape
# #

# Model

In [13]:
from transformers import ViTForImageClassification, ViTConfig


model = ViTForImageClassification(ViTConfig(
    num_labels=10,
    image_size=64,
    num_channels=12
)).to("cuda")

# Number of trainable parameters
sum(p.numel() for p in model.parameters() if p.requires_grad)

87437578

Load previous Model

In [14]:
# model = model.load_state_dict(torch.load("04162024-143828.pt"))

# Setup Cross Validation (adjusted)


In [15]:
from torch.utils.data import ConcatDataset, DataLoader, Subset
from sklearn.model_selection import KFold

# Assuming train_dataset and val_dataset_subset are your datasets
combined_train_val_dataset = ConcatDataset([train_dataset_subset, val_dataset_subset])

# Define the number of splits for cross-validation
k_folds = 2
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)


# Train

In [16]:
from datetime import datetime

execution_start = datetime.now().strftime("%m%d%Y-%H%M%S")

In [17]:
import torch
from tqdm import tqdm
from torch import nn, optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cross_entropy = nn.CrossEntropyLoss()
learning_rate = 0.005
num_epochs = 18

optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
model.train()


for fold, (train_ids, test_ids) in enumerate(kfold.split(combined_train_val_dataset)):
    print(f"Fold {fold+1}/{k_folds}")

    # Create data loaders for the current fold
    train_subsampler = Subset(combined_train_val_dataset, train_ids)
    val_subsampler = Subset(combined_train_val_dataset, test_ids)

    train_dataloader = DataLoader(train_subsampler, batch_size=64, shuffle=True)
    val_dataloader = DataLoader(val_subsampler, batch_size=64, shuffle=True)

    train_losses = []
    train_epoch_losses = []

    for epoch in range(num_epochs):
        train_mini_batch_losses = []
        moving_avg_loss = 0.0

        for x, y_dict in (pbar := tqdm(train_dataloader, ncols=10)):

            pbar.set_description(f"{moving_avg_loss: .3f}")
            x = x.float().to(device)
            y = y_dict['ID'].to(device)

            outputs = model(x)
            loss = cross_entropy(outputs.logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Store and log mini-batch losses
            train_mini_batch_losses.append(loss.item())
            train_losses.append(loss.item())
            moving_avg_loss = np.mean(train_losses[-5:])

        else:
            epoch_loss = np.mean(train_mini_batch_losses)
            pbar.set_description(f"{epoch_loss: .3f}")
            train_epoch_losses.append(epoch_loss)

        if epoch_loss < 0.75:
            break

    model.eval()
    validation_losses = []
    with torch.no_grad():
        for x, y_dict in val_dataloader:
            x = x.float().to(device)
            y = y_dict['ID'].to(device)
            outputs = model(x)
            loss = cross_entropy(outputs.logits, y)
            validation_losses.append(loss.item())

    avg_val_loss = np.mean(validation_losses)
    print(f"Validation Loss for Fold {fold+1}: {avg_val_loss:.3f}")

# Save final model
torch.save(model.state_dict(), f"{execution_start}.pt")

Fold 1/2


 0.000:   


KeyboardInterrupt: 

In [ ]:
import seaborn as sns


p = sns.lineplot(train_losses)
p.set(yscale='log')

# Validation

In [ ]:
# # set model in evaluation mode
# model.eval()


# validate = []
# for i, (imgs, labels) in enumerate(val_dataloader):
#     preds = model(imgs.to("cuda")).logits
#     pred_labels = [oh2text(oh.detach().cpu()) for oh in preds]
#     validate += list(zip(labels, preds))


# # TODO: Test Metrics

# Test

evaluation loop

In [ ]:
model.eval()

classArray = []

for i, (imgs, paths) in enumerate(test_dataloader):

    img_ids = [get_id(path) for path in paths]
    preds = model(imgs.to("cuda"))
    preds = list(preds.logits.detach())
    preds = [oh2text(pred.detach().cpu()) for pred in preds]
    classArray += list(zip(img_ids, preds))


## Submission





In [ ]:
import csv

with open(f'{execution_start}.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['test_id', 'label'])

    # Write each string to a row with its corresponding index as the test_id
    for i, label in classArray:
        writer.writerow([i, label])